Connect to Snowflake and create a file system

In [6]:
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session
session = Session.builder.configs(SnowflakeLoginOptions("test_conn")).create()

# server-side encrypted stage!
STAGE = "@TEST.PUBLIC.ENC_STAGE"

Create another file systems and list all files in the stage

In [7]:
from snowflake.ml.fileset import sfcfs
fs = sfcfs.SFFileSystem(snowpark_session=session)
fs.ls(STAGE)

['@TEST.PUBLIC.ENC_STAGE/imdb_train.csv.gz',
 '@TEST.PUBLIC.ENC_STAGE/titanic.csv',
 '@TEST.PUBLIC.ENC_STAGE/weather-ts.parquet']

Show CSV text data

In [8]:
with fs.open(f'{STAGE}/titanic.csv', mode='r') as f:
    print(f.read(1000))

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,,S
6,0,3,"Moran, Mr. James",male,,0,0,330877,8.4583,,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.075,,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,,S
10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,,C
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4,1,1,PP 9549,16.7,G6,S
12,1,1,"Bonnell, Miss. Elizabeth",female,58,0,0,113783,26.55,C103,S
13,0,3,"Saundercock, Mr. William Henry",male,20,0,0,A/

Show GZIP compresed data

In [9]:
import gzip
with fs.open(f'{STAGE}/imdb_train.csv.gz', mode='rb') as f:
    g = gzip.GzipFile(fileobj=f)
    for i in range(10):
        print(g.readline())

b'REVIEW,SENTIMENT\n'
b'"""In Victorian times a father is separated from his family when he is falsely accused of treason and they are sent to live in the country. The children adapt to their new situation, make friends, and enlist the help of a kind old man they wave to on the train to help reunite their family.<br /><br />Actors who direct movies are often not very good at it. Jeffries however, the great veteran actor of dozens of British comedy classics, is one of the few exceptions. His brilliant conception (he also wrote the script, from the novel by E. Nesbit) of a classic British children\'s story is what raises this film to art. Whilst the story may be highly idealised, the wonderful performances and the fabulously evocative Yorkshire dales settings combine to make a truly memorable movie. The photography by Arthur Ibbetson is the definition of good movie-making - not a shot is wasted in telling the story but at the same time the images combine to create a fabulously romantic a

Show Parquet data

In [10]:
import pyarrow.parquet as pq
df = pq.read_table(f'{STAGE}/weather-ts.parquet', filesystem=fs).to_pandas()
df

,INDEX,DATE,NAME,TEMP,WIND
0,4942,2022-05-12,THE BATTERY,50.0,0
1,4943,2022-05-12,STURGEON POINT,40.5,0
2,4944,2022-05-12,ROCHESTER,39.9,13.9
3,4945,2022-05-12,STEWART INTERNATIONAL AIRPORT,42.8,9.2
4,4946,2022-05-12,WESTCHESTER COUNTY AIRPORT,43.6,13.4
...,...,...,...,...,...
78315,83257,2018-01-01,GREATER ROCHESTER INTERNATION,32.0,8.9
78316,83258,2018-01-01,WHEELER-SACK AAF AIRPORT,31.4,6.4
78317,83259,2018-01-01,SULLIVAN COUNTY INTL ARPT,31.6,8.6
78318,83260,2018-01-01,WESTCHESTER COUNTY AIRPORT,38.2,8.7
